In [7]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [4]:
cuda = False
torch.manual_seed(42)
features = np.load("sider_latent.npy")

X_train, X_test = train_test_split(features, test_size=0.2, random_state = 42)

In [20]:
class LatentDataset(Dataset):
    def __init__(self,d):
        self.d = d
    def __getitem__(self,i):
        return  [self.d[i], self.d[i]]
    def __len__(self):
        return len(self.d)

class NoisyDataset(Dataset):
    def __init__(self,d,cc=100, downscale=0.25):
        self.data = d.copy()
        for i in range(1,cc): 
            self.data = np.concatenate((self.data, d.copy()),axis=0)
        self.noisy = self.data + np.asarray(np.random.randn(self.data.shape[0],self.data.shape[1]) * downscale , dtype=np.float32) 
    def __getitem__(self,i):
        return  [self.noisy[i], self.data[i]]
    def __len__(self):
        return len(self.data)


n_dim = features.shape[1]
internal_dim = 28
latent_dim = 10

dec = [10,n_dim]

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(n_dim, internal_dim)
        self.fc21 = nn.Linear(internal_dim, latent_dim)
        self.fc22 = nn.Linear(internal_dim, latent_dim)  
        self.fc3 = nn.Linear(latent_dim, internal_dim)
        self.fc4 = nn.Linear(internal_dim, n_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar



# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, target, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [26]:
train_loader = DataLoader(LatentDataset(X_train), batch_size=32, shuffle=True)
test_loader = DataLoader(LatentDataset(X_test), batch_size=32, shuffle=True)

In [35]:
device = torch.device( "cpu")
log_interval = 100
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(2):
    train(epoch)
    test(epoch)
    
with torch.no_grad():
    sample = torch.randn(1000, latent_dim).to(device)
    sample = model.decode(sample).cpu()
    np.save("synthetic.latent.npy",sample)

Train Epoch: 0 [0/614 (0%)]	Loss: 42.900997
====> Epoch: 0 Average loss: 36.8127
====> Test set loss: 28.8031
Train Epoch: 1 [0/614 (0%)]	Loss: 28.876337
====> Epoch: 1 Average loss: 18.0749
====> Test set loss: 0.1062


In [36]:
sample[2]

tensor([0.6231, 0.4248, 0.4319, 0.3758, 0.5609, 0.5628, 0.3988, 0.4671, 0.3798,
        0.5783, 0.5347, 0.4993, 0.6300, 0.4267, 0.4551, 0.4802, 0.4301, 0.4121,
        0.6392, 0.4903, 0.4137, 0.6724, 0.5549, 0.4031, 0.6350, 0.3587, 0.5829,
        0.4087, 0.4307, 0.3871, 0.4340, 0.4895, 0.4482, 0.4281, 0.4906, 0.3603,
        0.3312, 0.5338, 0.4016, 0.4096, 0.3442, 0.4470, 0.3658, 0.3492, 0.4992,
        0.3589, 0.5038, 0.3919, 0.3648, 0.4933, 0.3625, 0.3923, 0.4455, 0.5066,
        0.4159, 0.4520])